In [111]:
import pandas as pd
import numpy as np
import math
import scipy.stats as st

In [112]:
# Load dataset
data = pd.read_csv('titanic_data.csv')
print(data.describe())
print('- - - - - - - - - - - - - - - - - - - -')

         Survived      Pclass         Sex         Age  \
count  887.000000  887.000000  887.000000  887.000000   
mean     0.385569    2.305524    0.354002   29.471443   
std      0.487004    0.836662    0.478480   14.121908   
min      0.000000    1.000000    0.000000    0.420000   
25%      0.000000    2.000000    0.000000   20.250000   
50%      0.000000    3.000000    0.000000   28.000000   
75%      1.000000    3.000000    1.000000   38.000000   
max      1.000000    3.000000    1.000000   80.000000   

       Siblings/Spouses Aboard  Parents/Children Aboard       Fare  
count               887.000000               887.000000  887.00000  
mean                  0.525366                 0.383315   32.30542  
std                   1.104669                 0.807466   49.78204  
min                   0.000000                 0.000000    0.00000  
25%                   0.000000                 0.000000    7.92500  
50%                   0.000000                 0.000000   14.45420  
75%

In [113]:
# Variable transform
CATEGORICAL_FEATURES = ['Pclass', 'Sex', 'Siblings/Spouses Aboard', 'Parents/Children Aboard']
NUMERICAL_FEATURES = ['Age', 'Fare']
for feature_name in CATEGORICAL_FEATURES:
    data = data.join(pd.get_dummies(data[feature_name], prefix=feature_name))
    data.drop([feature_name], axis = 1, inplace=True)
for feature_name in NUMERICAL_FEATURES:
    data = data.join(pd.get_dummies(pd.qcut(data[feature_name], 5), prefix=feature_name))
    data.drop([feature_name], axis = 1, inplace=True)
print('First, I divided these six features into two categories.')
print('CATEGORICAL_FEATURES and NUMERICAL_FEATURES')
print('For CATEGORICAL_FEATURES, I directly convert it to one-hot encoding.')
print('For NUMERICAL_FEATURES, binning the data and then convert it to one-hot encoding.')
print(data.describe())
print('- - - - - - - - - - - - - - - - - - - -')

First, I divided these six features into two categories.
CATEGORICAL_FEATURES and NUMERICAL_FEATURES
For CATEGORICAL_FEATURES, I directly convert it to one-hot encoding.
For NUMERICAL_FEATURES, binning the data and then convert it to one-hot encoding.
         Survived    Pclass_1    Pclass_2    Pclass_3       Sex_0       Sex_1  \
count  887.000000  887.000000  887.000000  887.000000  887.000000  887.000000   
mean     0.385569    0.243517    0.207441    0.549042    0.645998    0.354002   
std      0.487004    0.429447    0.405703    0.497870    0.478480    0.478480   
min      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
25%      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
50%      0.000000    0.000000    0.000000    1.000000    1.000000    0.000000   
75%      1.000000    0.000000    0.000000    1.000000    1.000000    1.000000   
max      1.000000    1.000000    1.000000    1.000000    1.000000    1.000000   

       Siblings/Sp

In [110]:
# Mutual Information

# Conditional Probability
def Pr(FEATURE, val, ):
    